In [92]:
import nltk
from nltk.tokenize import NLTKWordTokenizer, PunktSentenceTokenizer
import spacy
nlp = spacy.load("ru_core_news_sm")
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from collections import Counter
from nltk.corpus import stopwords

In [151]:
#Дополняем стоп слова
stopWords = stopwords.words("russian")
add_stop = []
stopWords.extend(['её', 'неё'])
for word in stopWords:
    add_stop.append(word.title())
stopWords.extend(add_stop)
#stopWords

In [149]:
class Analysis:

    def __init__(self, text:str):
        self.text = text

    def tokenize(self) -> list[str]:
        tokenizer = NLTKWordTokenizer()
        tokens = tokenizer.tokenize(text)
        removing = [' ', '•', '.', ',', '?', '!', ':', ';', '—', '«', '»', ')', '(', '[', ']', '...', '-', '–', '``', "''", 'note', '..']
        for symb in removing:
            while symb in tokens:
                tokens.remove(symb)
        return tokens
    
    def clear_stops(self) -> list[str]:
        clear_tokens = self.tokenize()
        for word in stopWords:
            while word in clear_tokens:
                clear_tokens.remove(word)
        return clear_tokens

    def lex_diverse(self):
        tokens = self.clear_stops()
        list_len = len(tokens)
        set_len = len(set(tokens))
        diversity = set_len / list_len
        return f'Лексическое разнообразие: {round((diversity), 3)}'
    
    def quantify(self):
        pos_list = []
        for word in self.tokenize():
            pos_list.append(morph.parse(word)[0].tag.POS)
        pos_counter = Counter(pos_list)
        top_5_pos = pos_counter.most_common(5)

        word_counter = Counter(self.clear_stops())
        top_20_words = word_counter.most_common(20)
        return f"Топ 5 частей речи:{top_5_pos}\n\tТоп 20 слов: {top_20_words}"
    
    def av_sent_len(self):
        pst = PunktSentenceTokenizer()
        sentences = pst.tokenize(text)
        lengths = []
        for sent in sentences:
            lengths.append(len(sent))
        return f'Average sentence length is {round(sum(lengths)/len(lengths), 2)}'
    
    def dynamics(self):
        pst = PunktSentenceTokenizer()
        sentences = pst.tokenize(text)
        tokenizer = NLTKWordTokenizer()
        dynamic = []
        for sent in sentences:
            sent_tokens = tokenizer.tokenize(sent)
            verbs = 0
            for word in sent_tokens:
                pos = morph.parse(word)[0].tag.POS
                if pos == 'VERB' or pos == 'INFN':
                    verbs += 1
            sent_dynamic = verbs / len(sent)
            dynamic.append(sent_dynamic)
        return f'Среднее соотношение глаголов к словам в предложении (АКА Динамика текста) = {round(sum(dynamic)/len(dynamic),4)}'
    
    def literary(self):
        pst = PunktSentenceTokenizer()
        sentences = pst.tokenize(text)
        tokenizer = NLTKWordTokenizer()
        literarity = []
        for sent in sentences:
            sent_tokens = tokenizer.tokenize(sent)
            descript = 0
            for word in sent_tokens:
                pos = morph.parse(word)[0].tag.POS
                if pos == 'ADJF' or pos == 'ADJS' or pos == 'PRTF' or pos == 'PRTS' or pos == 'ADVB' or pos == 'GRND':
                    descript += 1
            sent_dynamic = descript / len(sent)
            literarity.append(sent_dynamic)
        return f'Среднее соотношение описывающих частей речи к словам в предложении (АКА Художественное наполнение) = {round(sum(literarity)/len(literarity),4)}'

            


In [150]:
texts = ['breter_1846.txt', 'burmistr_1847.txt', 'mumu_1854.txt', 'asya_1858.txt', 'otcy_i_deti_1862.txt', 'korol_lir_1870.txt']

for el in texts:
    with open(el, 'r') as file:
        text = file.read()
    book = Analysis(text)
    print(f"""КНИГА {el}
        {book.lex_diverse()}
        {book.quantify()}
        {book.av_sent_len()}
        {book.dynamics()}
        {book.literary()}
----------------------------------
    """)

КНИГА breter_1846.txt
        Лексическое разнообразие: 0.595
        Топ 5 частей речи:[('NOUN', 2516), ('VERB', 1943), (None, 1366), ('NPRO', 1348), ('PREP', 1317)]
	Топ 20 слов: [('Маша', 109), ('Кистер', 99), ('Лучков', 74), ('Авдей', 53), ('Ненила', 35), ('человек', 34), ('Макарьевна', 30), ('Фёдор', 28), ('всё', 25), ('Кистера', 24), ('очень', 21), ('Фёдорович', 19), ('вами', 19), ('это', 19), ('Кистер.', 18), ('Иванович', 17), ('своей', 17), ('тебе', 17), ('Маша.', 17), ('своего', 16)]
        Average sentence length is 71.59
        Среднее соотношение глаголов к словам в предложении (АКА Динамика текста) = 0.0304
        Среднее соотношение описывающих частей речи к словам в предложении (АКА Художественное наполнение) = 0.0277
----------------------------------
    
КНИГА burmistr_1847.txt
        Лексическое разнообразие: 0.757
        Топ 5 частей речи:[('NOUN', 889), ('VERB', 478), ('PREP', 387), ('ADJF', 368), ('CONJ', 357)]
	Топ 20 слов: [('Аркадий', 46), ('Павлыч', 41), 

## Выводы

Проведя сравнение текстов, можно заметить что на позднее начало - середину творческтва (1847 - 1858гг) приходятся наиболее лексически разнообразные работы. При том что средняя длина предложений остается в них наиболее высокой.

Динамика текста и его Художественное наполнение остаются примерно на одном уровне в течении всей карьеры (варьируясь между произведениями)

Что не удивительно
Наиболее используемыми частями речи являются глагол и существительное.
А наиболее часто используемые слова - это имена персонажей. либо способы обращения к ним (наприм: собака, барышня, вами)